# Install packages 

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Setting to make numbers easier to read on display
pd.options.display.float_format = '{:20.2f}'.format

# Show all columns on output
pd.set_option('display.max_columns', 999)

# Importing excel data set and do some data exploration
? what if i want to connect to a dwh query?

In [26]:
df = pd.read_excel('online_retail_II.xlsx', sheet_name= 0)
# Reading data from excel file and storing it in dataframe df. Only read sheet 0.
# What is next? Reading the other sheet and merging them.

In [27]:
df.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.00,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.00,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.00,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.00,United Kingdom


In [28]:
df.info()
# Checking data types and non-null counts. We expect the count of each column to be 541909 as per the data source description.
# We can see that there are some missing values in the columns Customer ID and Description.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [29]:
df.describe()
# Automatically shows only numerical columns.
# Getting a statistical summary of the numerical columns in the dataframe.
# We can see that there are some negative values in the Quantity and Price columns which we need to investigate further.

,Quantity,InvoiceDate,Price,Customer ID
count,525461.00,525461,525461.00,417534.00
mean,10.34,2010-06-28 11:37:36.845017856,4.69,15360.65
min,-9600.00,2009-12-01 07:45:00,-53594.36,12346.00
25%,1.00,2010-03-21 12:20:00,1.25,13983.00
50%,3.00,2010-07-06 09:51:00,2.10,15311.00
75%,10.00,2010-10-15 12:45:00,4.21,16799.00
max,19152.00,2010-12-09 20:01:00,25111.09,18287.00
std,107.42,NaN,146.13,1680.81


In [30]:
# To include all the OBJECT columns in the output of describe method. Object is string type in pandas.
df.describe(include='O')

,Invoice,StockCode,Description,Country
count,525461,525461,522533,525461
unique,28816,4632,4681,40
top,537434,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,675,3516,3549,485852


In [31]:
df[df["Customer ID"].isna()].head(10)
# Checking the rows with missing Customer ID values. We can see that these are mostly transactions with negative Quantity values which indicates that these are probably returns.]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.00,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,NaN,United Kingdom
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,NaN,United Kingdom
578,489525,85227,SET/6 3D KIT CARDS FOR KIDS,1,2009-12-01 11:49:00,0.85,NaN,United Kingdom
1055,489548,22271,FELTCRAFT DOLL ROSIE,1,2009-12-01 12:32:00,2.95,NaN,United Kingdom
1056,489548,22254,FELT TOADSTOOL LARGE,12,2009-12-01 12:32:00,1.25,NaN,United Kingdom
1057,489548,22273,FELTCRAFT DOLL MOLLY,3,2009-12-01 12:32:00,2.95,NaN,United Kingdom
1058,489548,22195,LARGE HEART MEASURING SPOONS,1,2009-12-01 12:32:00,1.65,NaN,United Kingdom


In [32]:
df[df["Quantity"] < 0].head(10)
# Checking the rows with negative Quantity values. We can see that these are mostly transactions with missing Customer ID values which indicates that these are probably returns.

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.00,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.00,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.00,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.00,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.00,Australia
183,C489449,21871,SAVE THE PLANET MUG,-12,2009-12-01 10:33:00,1.25,16321.00,Australia
184,C489449,84946,ANTIQUE SILVER TEA GLASS ETCHED,-12,2009-12-01 10:33:00,1.25,16321.00,Australia
185,C489449,84970S,HANGING HEART ZINC T-LIGHT HOLDER,-24,2009-12-01 10:33:00,0.85,16321.00,Australia
186,C489449,22090,PAPER BUNTING RETRO SPOTS,-12,2009-12-01 10:33:00,2.95,16321.00,Australia
196,C489459,90200A,PURPLE SWEETHEART BRACELET,-3,2009-12-01 10:44:00,4.25,17592.00,United Kingdom


C means Cancelation here. Check the data description.
So, we actually want to check if the Invoice Code only contains the C and 6 digits, or it also contains other characters. Dont just blindly believe in the docu, duh.
To do it, the first step is to convert Invoice Id into str.

In [33]:
df["Invoice"] = df["Invoice"].astype(str)
# Convert Invoice column to string type to check its format.
df["Invoice"].str.match(r'^C\d{6}$').all()
# Check if all Invoice values match the pattern of starting with 'C' followed by exactly 6 digits.
# The result is False, indicating that there are Invoice values that do not match this pattern.

np.False_

So.. what can it be?? if not C, then what? 
We would want to use unique to see all the possible options.

In [34]:
df["Invoice"].str.replace("[0-9]", "", regex=True).unique()
# We see there are Invoice which starts with A. Let's check what it is.

array(['', 'C', 'A'], dtype=object)

In [35]:
df[df["Invoice"].str.startswith('A')].head(10)
# Checking the rows with Invoice starting with 'A'. Only 3 rows found.
# We can see in the Description column that these are transactions related to adjustments or corrections to previous invoices.

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,NaN,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,NaN,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,NaN,United Kingdom


Since this information is irrelevant to our goal of customer segmentation, we can delete these three rows.

In [36]:
df["StockCode"] = df["StockCode"].astype(str)
# Convert StockCode column to string type to check its format.

In [37]:
df["StockCode"].str.match(r'^\d{5}$').all()
# Check if all StockCode values match the pattern of exactly 5 digits as in the docu.
# The result is False, indicating that there are StockCode values that do not match this pattern.
# Let's take a look at some samples

np.False_

In [38]:
df[df["StockCode"].str.match(r'^\d{5}$') == False].head(10)
# Checking the rows with StockCode values that do not match the pattern of exactly 5 digits

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
12,489436,48173C,DOOR MAT BLACK FLOCK,10,2009-12-01 09:06:00,5.95,13078.00,United Kingdom
23,489436,35004B,SET OF 3 BLACK FLYING DUCKS,12,2009-12-01 09:06:00,4.65,13078.00,United Kingdom
28,489436,84596F,SMALL MARSHMALLOWS PINK BOWL,8,2009-12-01 09:06:00,1.25,13078.00,United Kingdom
29,489436,84596L,BISCUITS SMALL BOWL LIGHT BLUE,8,2009-12-01 09:06:00,1.25,13078.00,United Kingdom
42,489437,84507B,STRIPES DESIGN MONKEY DOLL,6,2009-12-01 09:08:00,2.55,15362.00,United Kingdom
46,489437,84970S,HANGING HEART ZINC T-LIGHT HOLDER,12,2009-12-01 09:08:00,0.85,15362.00,United Kingdom
61,489438,84031A,CHARLIE+LOLA RED HOT WATER BOTTLE,56,2009-12-01 09:24:00,3.00,18102.00,United Kingdom
62,489438,84031B,CHARLIE LOLA BLUE HOT WATER BOTTLE,56,2009-12-01 09:24:00,3.00,18102.00,United Kingdom


In [39]:
df[(df["StockCode"].str.match(r'^\d{5}$') == False) & (df["StockCode"].str.match("^\d{5}[a-zA-Z]+$") == False)]["StockCode"].unique()
# Checking the StockCode values that DO NOT match the pattern of exactly 5 digits or 5 digits followed by letters.
# We can see that there are StockCode values like 'POST', 'D', 'M', 'S', 'BANK CHARGES', 'DOT', 'PADS', 'CRUK', 'AMAZONFEE', etc.

array(['POST', 'D', 'DCGS0058', 'DCGS0068', 'DOT', 'M', 'DCGS0004',
       'DCGS0076', 'C2', 'BANK CHARGES', 'DCGS0003', 'TEST001',
       'gift_0001_80', 'DCGS0072', 'gift_0001_20', 'DCGS0044', 'TEST002',
       'gift_0001_10', 'gift_0001_50', 'DCGS0066N', 'gift_0001_30',
       'PADS', 'ADJUST', 'gift_0001_40', 'gift_0001_60', 'gift_0001_70',
       'gift_0001_90', 'DCGSSGIRL', 'DCGS0006', 'DCGS0016', 'DCGS0027',
       'DCGS0036', 'DCGS0039', 'DCGS0060', 'DCGS0056', 'DCGS0059', 'GIFT',
       'DCGSLBOY', 'm', 'DCGS0053', 'DCGS0062', 'DCGS0037', 'DCGSSBOY',
       'DCGSLGIRL', 'S', 'DCGS0069', 'DCGS0070', 'DCGS0075', 'B',
       'DCGS0041', 'ADJUST2', '47503J ', 'C3', 'SP1002', 'AMAZONFEE'],
      dtype=object)

We have many questions, and this list doesn't include any documented values. This means we have to look closely at each code to make an assumption.

In [40]:
df[df["StockCode"].str.contains("POST") | df["StockCode"].str.contains("DOT")].head(10)
# example checking rows with StockCode containing 'POST' or 'DOT'

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
89,489439,POST,POSTAGE,3,2009-12-01 09:28:00,18.00,12682.00,France
126,489444,POST,POSTAGE,1,2009-12-01 09:55:00,141.00,12636.00,USA
173,489447,POST,POSTAGE,1,2009-12-01 10:10:00,130.00,12362.00,Belgium
625,489526,POST,POSTAGE,6,2009-12-01 11:50:00,18.00,12533.00,Germany
927,C489538,POST,POSTAGE,-1,2009-12-01 12:18:00,9.58,15796.00,United Kingdom
1244,489557,POST,POSTAGE,4,2009-12-01 12:52:00,18.00,12490.00,France
2379,489597,DOT,DOTCOM POSTAGE,1,2009-12-01 14:28:00,647.19,NaN,United Kingdom
2539,489600,DOT,DOTCOM POSTAGE,1,2009-12-01 14:43:00,55.96,NaN,United Kingdom
2551,489601,DOT,DOTCOM POSTAGE,1,2009-12-01 14:44:00,68.39,NaN,United Kingdom
2571,489602,DOT,DOTCOM POSTAGE,1,2009-12-01 14:45:00,59.35,NaN,United Kingdom


## Stock Code Results
which i copied from the autor directly
Source: https://www.youtube.com/watch?v=afPJeQuVeuY

| Code         | Description                                                                 | Action                   |
|--------------|-----------------------------------------------------------------------------|--------------------------|
| DCGS         | Looks valid, some quantities are negative though and customer ID is null     | Exclude from clustering  |
| D            | Looks valid, represents discount values                                     | Exclude from clustering  |
| DOT          | Looks valid, represents postage charges                                     | Exclude from clustering  |
| M or m       | Looks valid, represents manual transactions                                 | Exclude from clustering  |
| C2           | Carriage transaction - not sure what this means                             | Exclude from clustering  |
| C3           | Not sure, only 1 transaction                                                | Exclude from clustering  |
| BANK CHARGES or B | Bank charges                                                           | Exclude from clustering  |
| S            | Samples sent to customer                                                    | Exclude from clustering  |
| TESTXXX      | Testing data, not valid                                                     | Exclude from clustering  |
| gift__XXX    | Purchases with gift cards, might be interesting for another analysis, but no customer data | Exclude         |
| PADS         | Looks like a legit stock code for padding                                   | Include                  |
| SP1002       | Looks like a special request item, only 2 transactions, 3 look legit, 1 has 0 pricing | Exclude for now  |
| AMAZONFEE    | Looks like fees for Amazon shipping or something                            | Exclude for now          |
| ADJUSTX      | Looks like manual account adjustments by admins                             | Exclude for now          |

In a real-world situation, this is where I will talk to a colleague and ask for more information to decide whether to keep the data or not.